In [20]:
import numpy as np
import fiona
import pprint
from madmex.models import PredictObject
from madmex.models import PredictClassification, Region
from madmex.util import chunk
import json
from shapely.geometry import shape, mapping
from rasterio.warp import transform_geom
from rasterio.crs import CRS as CRS_rio
from affine import Affine
from rasterio.features import rasterize
import rasterio
from dask.distributed import Client
from rasterio.merge import merge
import os
from operator import itemgetter
from fiona.crs import to_string
from rasterio import features

In [2]:
#name of prediction

In [3]:
name_predict = 'lc_rf_poly_500_2k_min_thr_gr_s1_2_20m_res_10m_jalisco_2017'

In [4]:
qs_ids = PredictClassification.objects.filter(name=name_predict).distinct('predict_object_id')

In [5]:
len(qs_ids)

56

In [6]:
list_ids = [x.predict_object_id for x in qs_ids]

In [7]:
region = 'Jalisco'

In [8]:
region_geom = Region.objects.get(name=region).the_geom
region_geojson = region_geom.geojson
geometry_region = json.loads(region_geojson)

In [35]:
def fun(id, path_fs, resolution, geometry_region):
    shape_Jalisco=shape(geometry_region)
    seg = PredictObject.objects.filter(id=id)
    path = seg[0].path
    poly = seg[0].the_geom
    poly_geojson = poly.geojson
    geometry_seg = json.loads(poly_geojson)
    segmentation_name_classified = path.split('/')[3].split('.')[0] + '_classified'
    with fiona.open(path) as src:
        crs = to_string(src.crs)
        shape_Jalisco_dc_tile = shape_Jalisco.intersection(shape(geometry_seg))
        geom_Jalisco_dc_tile_geojson = mapping(shape_Jalisco_dc_tile)
        pred_objects_sorted = PredictClassification.objects.filter(name=name_predict, predict_object_id=id).prefetch_related('tag').order_by('features_id')
        fc_pred=[(x['properties']['id'], x['geometry']) for x in src]
        fc_pred_sorted = sorted(fc_pred, key=itemgetter(0))
        fc_pred = [(x[0][1], 
                    x[1].tag.numeric_code) for x in zip(fc_pred_sorted, pred_objects_sorted)]
        fc_schema = {'geometry': 'Polygon',
                     'properties': {'code': 'int'}}
        #rasterize
        geometry_seg_proj = transform_geom(CRS.from_epsg(4326),
                                           CRS.from_proj4(crs),
                                           geometry_seg)
        xmin, ymin, xmax, ymax = shape(geometry_seg_proj).bounds
        nrows = int(((ymax - ymin) // resolution) + 1)
        ncols = int(((xmax - xmin) // resolution) + 1)
        shape_dim = (nrows, ncols)
        arr = np.zeros((nrows, ncols), dtype=np.uint8)
        aff = Affine(resolution, 0, xmin, 0, -resolution, ymax)
        rasterize(shapes=fc_pred, transform=aff, dtype=np.uint8, out=arr)
        meta = {'driver': 'GTiff',
                'width': shape_dim[1],
                'height': shape_dim[0],
                'count': 1,
                'dtype': arr.dtype,
                'crs': crs,
                'transform': aff,
                'compress': 'lzw',
                'nodata': 0}
        geometry_region_proj = transform_geom(CRS_rio.from_epsg(4326),
                                              CRS_rio.from_proj4(meta['crs']),
                                              geom_Jalisco_dc_tile_geojson)
        mask_array = features.rasterize(shapes=[(geometry_region_proj, 1)],
                                        out_shape=shape_dim,
                                        fill=0,
                                        transform=meta['transform'],
                                        dtype=rasterio.uint8)
        arr[mask_array==0] = 0
        filename_masked = path_fs + segmentation_name_classified + '.tif'
        with rasterio.open(filename_masked, "w", **meta) as dst:
            dst.write(arr,1)
        

    return filename_masked

In [10]:
path_fs = '/shared_volume/tasks/2019/fiona_cloud_usage/db_to_raster_rasterio_mask_parallel/'

In [11]:
if not os.path.exists(path_fs):
    os.makedirs(path_fs)

In [12]:
scheduler_file= '/shared_volume/scheduler.json'

In [13]:
client = Client(scheduler_file=scheduler_file)

In [14]:
client

Client Scheduler: tcp://100.96.2.93:8786 Dashboard: http://100.96.2.93:8787/status,Cluster Workers: 29 Cores: 29 Memory: 116.00 GB


In [36]:
client.restart()

Client Scheduler: tcp://100.96.2.93:8786 Dashboard: http://100.96.2.93:8787/status,Cluster Workers: 20 Cores: 20 Memory: 80.00 GB


In [37]:
resolution = 10
c = client.map(fun,list_ids,**{'path_fs': path_fs,
                              'resolution': resolution,
                              'geometry_region': geometry_region})

In [38]:
result = client.gather(c)

In [39]:
src_files_to_mosaic=[]

In [40]:
for file in result:
    src = rasterio.open(file)
    src_files_to_mosaic.append(src)

In [41]:
src.crs

CRS.from_wkt('PROJCS["unnamed",GEOGCS["unnamed ellipse",DATUM["unknown",SPHEROID["unnamed",6378137,6556752.316985585]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["standard_parallel_1",17.5],PARAMETER["standard_parallel_2",29.5],PARAMETER["latitude_of_origin",12],PARAMETER["central_meridian",-102],PARAMETER["false_easting",2500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]]]')

In [42]:
mosaic, out_trans = merge(src_files_to_mosaic,nodata=0)

In [43]:
meta = {'driver': 'GTiff',
        'width': mosaic.shape[2],
        'height': mosaic.shape[1],
        'count': 1,
        'dtype': mosaic.dtype,
        'crs': src.crs,
        'transform': out_trans,
        'compress': 'lzw',
        'nodata': 0}

In [44]:
meta

{'driver': 'GTiff',
 'width': 50022,
 'height': 45019,
 'count': 1,
 'dtype': dtype('uint8'),
 'crs': CRS.from_wkt('PROJCS["unnamed",GEOGCS["unnamed ellipse",DATUM["unknown",SPHEROID["unnamed",6378137,6556752.316985585]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["standard_parallel_1",17.5],PARAMETER["standard_parallel_2",29.5],PARAMETER["latitude_of_origin",12],PARAMETER["central_meridian",-102],PARAMETER["false_easting",2500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]]]'),
 'transform': Affine(10.0, 0.0, 2077599.999999996,
        0.0, -10.0, 1226240.000000003),
 'compress': 'lzw',
 'nodata': 0}

In [45]:
filename_mosaic = path_fs + 'mosaic_db_to_raster_parallel_mask.tif'

In [46]:
with rasterio.open(filename_mosaic, 'w', **meta) as dst:
    dst.write(mosaic)

In [47]:
#close & clean:
for ind in range(0,len(result)):
    src_files_to_mosaic[ind].close()
    os.remove(result[ind])